In [ ]:
-- Using Snowflake!

In [ ]:
-- Puzzle 1 DDL
CREATE OR REPLACE TEMPORARY TABLE Cart1
    (
    Item  VARCHAR(100) PRIMARY KEY
    )
;

CREATE OR REPLACE TEMPORARY TABLE Cart2
    (
    Item  VARCHAR(100) PRIMARY KEY
    )
;

INSERT INTO Cart1 (Item) VALUES
('Sugar'),('Bread'),('Juice'),('Soda'),('Flour')
;

INSERT INTO Cart2 (Item) VALUES
('Sugar'),('Bread'),('Butter'),('Cheese'),('Fruit')
;

In [ ]:
-- Puzzle 1
SELECT
    *
FROM
    cart1 AS c1
    FULL JOIN cart2 AS c2 ON c2.item=c1.item
;

In [ ]:
-- Puzzle 2 DDL
CREATE OR REPLACE TEMPORARY TABLE Employees
    (
    EmployeeID  INTEGER PRIMARY KEY,
    ManagerID   INTEGER NULL,
    JobTitle    VARCHAR(100) NOT NULL
    )
;

INSERT INTO Employees (EmployeeID, ManagerID, JobTitle)
VALUES
    (1001,NULL,'President')
    ,(2002,1001,'Director')
    ,(3003,1001,'Office Manager')
    ,(4004,2002,'Engineer')
    ,(5005,2002,'Engineer')
    ,(6006,2002,'Engineer')
;

In [ ]:
-- Puzzle 2 (Solution 1)
WITH emp_hierarchy AS
    (
    SELECT
        e.employeeid
        ,e.managerid
        ,e.jobtitle
        ,0 AS depth
    FROM
        employees AS e
    WHERE 1=1
        AND e.managerid IS NULL

    UNION ALL

    SELECT
        e.employeeid
        ,e.managerid
        ,e.jobtitle
        ,eh.depth+1 AS depth
    FROM
        emp_hierarchy AS eh
        JOIN employees AS e ON e.managerid=eh.employeeid
    )

SELECT
    *
FROM
    emp_hierarchy AS eh
ORDER BY
    eh.depth
;

In [ ]:
-- Puzzle 2 (Solution 2)
SELECT
    e.employeeid
    ,e.managerid
    ,e.jobtitle
    ,ARRAY_SIZE(SPLIT(SUBSTR(SYS_CONNECT_BY_PATH(e.jobtitle, '.'), 2, LENGTH(SYS_CONNECT_BY_PATH(e.jobtitle, '.'))), '.'))-1 AS depth
FROM
    employees AS e
    START WITH
        jobtitle = 'President'
    CONNECT BY
        managerid = PRIOR employeeid
ORDER BY
    employeeid
;

In [ ]:
-- Puzzle 3 DDL
CREATE OR REPLACE TEMPORARY TABLE EmployeePayRecords
    (
    EmployeeID  INTEGER,
    FiscalYear  INTEGER,
    StartDate   DATE,
    EndDate     DATE,
    PayRate     DECIMAL(38,2)
    )
;

In [ ]:
-- Puzzle 3
-- Note: Snowflake doesn't support SQL Server Check Constraints

ALTER TABLE EmployeePayRecords ADD CONSTRAINT unq_emp_fiscyear UNIQUE (employeeid, fiscalyear);

In [ ]:
-- Puzzle 4 DDL
CREATE OR REPLACE TEMPORARY TABLE Orders
    (
    CustomerID     INTEGER,
    OrderID        INTEGER,
    DeliveryState  VARCHAR(100) NOT NULL,
    Amount         DECIMAL(38,2) NOT NULL
    )
;

INSERT INTO Orders (CustomerID, OrderID, DeliveryState, Amount)
VALUES
    (1001,1,'CA',340)
    ,(1001,2,'TX',950)
    ,(1001,3,'TX',670)
    ,(1001,4,'TX',860)
    ,(2002,5,'WA',320)
    ,(3003,6,'CA',650)
    ,(3003,7,'CA',830)
    ,(4004,8,'TX',120)
;

In [ ]:
-- Puzzle 4
SELECT
    *
FROM
    orders AS o
WHERE 1=1
    AND o.deliverystate='TX'
    AND EXISTS(
                SELECT
                    1
                FROM
                    orders AS o2
                WHERE 1=1
                    AND o2.customerid=o.customerid
                    AND o2.deliverystate='CA'
                )
;

In [ ]:
-- Puzzle 5 DDL
CREATE OR REPLACE TEMPORARY TABLE PhoneDirectory
    (
    CustomerID   INTEGER,
    Type       VARCHAR(100),
    PhoneNumber  VARCHAR(12) NOT NULL
    )
;

INSERT INTO PhoneDirectory (CustomerID, Type, PhoneNumber)
VALUES
    (1001,'Cellular','555-897-5421')
    ,(1001,'Work','555-897-6542')
    ,(1001,'Home','555-698-9874')
    ,(2002,'Cellular','555-963-6544')
    ,(2002,'Work','555-812-9856')
    ,(3003,'Cellular','555-987-6541')
;

In [ ]:
-- Puzzle 5 (Solution 1 - Clean)
SELECT
    p.customerid
    ,p."'Cellular'" AS cellular
    ,p."'Work'" AS work
    ,p."'Home'" AS home
FROM
    phonedirectory AS pd
    PIVOT (MAX(phonenumber) FOR type IN (ANY ORDER BY type)) AS p
;

In [ ]:
-- Puzzle 5 (Solution 2 - Hacky)
SELECT
    pd.customerid
    ,MAX(IFF(pd.type='Cellular', pd.phonenumber, NULL)) AS cellular
    ,MAX(IFF(pd.type='Work', pd.phonenumber, NULL)) AS work
    ,MAX(IFF(pd.type='Home', pd.phonenumber, NULL)) AS home
FROM
    phonedirectory AS pd
GROUP BY ALL
ORDER BY
    pd.customerid
;

In [ ]:
-- Puzzle 6 DDL
CREATE OR REPLACE TEMPORARY TABLE WorkflowSteps
    (
    Workflow        VARCHAR(100),
    StepNumber      INTEGER,
    CompletionDate  DATE NULL
    )
;

INSERT INTO WorkflowSteps (Workflow, StepNumber, CompletionDate)
VALUES
    ('Alpha',1,'7/2/2018')
    ,('Alpha',2,'7/2/2018')
    ,('Alpha',3,'7/1/2018')
    ,('Bravo',1,'6/25/2018')
    ,('Bravo',2,NULL)
    ,('Bravo',3,'6/27/2018')
    ,('Charlie',1,NULL)
    ,('Charlie',2,'7/1/2018')
;

In [ ]:
-- Puzzle 6 (Solution 1)
SELECT
    ws.workflow
FROM
    workflowsteps AS ws
WHERE 1=1
    AND completiondate IS NULL
;

In [ ]:
-- Puzzle 6 (Solution 2 - Bonus)
SELECT
    ws.workflow
FROM
    workflowsteps AS ws
GROUP BY
    ws.workflow
HAVING
    COUNT(*) <> COUNT(ws.completiondate)
;

In [ ]:
-- Puzzle 7 DDL
CREATE OR REPLACE TEMPORARY TABLE Candidates
    (
    CandidateID  INTEGER,
    Occupation   VARCHAR(100)
    )
;

INSERT INTO Candidates (CandidateID, Occupation)
VALUES
    (1001,'Geologist')
    ,(1001,'Astrogator')
    ,(1001,'Biochemist')
    ,(1001,'Technician')
    ,(2002,'Surgeon')
    ,(2002,'Machinist')
    ,(2002,'Geologist')
    ,(3003,'Geologist')
    ,(3003,'Astrogator')
    ,(4004,'Selenologist')
;

CREATE OR REPLACE TEMPORARY TABLE Requirements
    (
    Requirement  VARCHAR(100)
    )
;

INSERT INTO Requirements (Requirement)
    VALUES
    ('Geologist')
    ,('Astrogator')
    ,('Technician')
;

In [ ]:
-- Puzzle 7
;WITH count_requirements AS
    (
    SELECT
        COUNT(*) AS ct
    FROM
        requirements AS r
    )
    
SELECT
    c.candidateid
    ,cr.ct
FROM
    candidates AS c
    JOIN requirements AS r ON r.requirement=c.occupation
    CROSS JOIN count_requirements AS cr
GROUP BY ALL
HAVING
    COUNT(*)=cr.ct
;

In [ ]:
-- Puzzle 8 DDL
CREATE OR REPLACE TEMPORARY TABLE WorkflowCases
    (
    Workflow  VARCHAR(100),
    Case1     INTEGER NOT NULL DEFAULT 0,
    Case2     INTEGER NOT NULL DEFAULT 0,
    Case3     INTEGER NOT NULL DEFAULT 0
    )
;

INSERT INTO WorkflowCases (Workflow, Case1, Case2, Case3)
VALUES
    ('Alpha',0,0,0)
    ,('Bravo',0,1,1)
    ,('Charlie',1,0,0)
    ,('Delta',0,0,0)
;

In [ ]:
-- Puzzle 8 (Solution 1 - Clean)
SELECT
    up.workflow
    ,SUM(up.passed) AS passed
FROM
    workflowcases AS wc
    UNPIVOT(passed FOR case IN (Case1, Case2, Case3)) AS up
GROUP BY ALL
;

In [ ]:
-- Puzzle 8 (Solution 2 - Hacky)
SELECT
    wc.workflow
    ,wc.case1+wc.case2+wc.case3 AS passed
FROM
    workflowcases AS wc
;

In [ ]:
-- Puzzle 9 DDL
CREATE OR REPLACE TEMPORARY TABLE Employees
    (
    EmployeeID  INTEGER,
    License     VARCHAR(100)
    )
;

INSERT INTO Employees (EmployeeID, License)
VALUES
    (1001,'Class A')
    ,(1001,'Class B')
    ,(1001,'Class C')
    ,(2002,'Class A')
    ,(2002,'Class B')
    ,(2002,'Class C')
    ,(3003,'Class A')
    ,(3003,'Class D')
    ,(4004,'Class A')
    ,(4004,'Class B')
    ,(4004,'Class D')
    ,(5005,'Class A')
    ,(5005,'Class B')
    ,(5005,'Class D')
;

In [ ]:
-- Puzzle 9
;WITH employees_enhanced AS
    (
    SELECT
        *
        ,COUNT(*) OVER(PARTITION BY e.employeeid) AS ct_employee_licenses
    FROM
        employees AS e
    )
    
SELECT DISTINCT
    e.employeeid
    ,e2.employeeid
    ,COUNT(*) OVER(PARTITION BY e.employeeid, e2.employeeid) AS ct_combo_licenses
FROM
    employees_enhanced AS e
    JOIN employees_enhanced AS e2 ON e2.employeeid<>e.employeeid AND e2.license=e.license
WHERE 1=1
    AND e.ct_employee_licenses=e2.ct_employee_licenses
QUALIFY
    e.ct_employee_licenses=ct_combo_licenses
ORDER BY
    e.employeeid
    ,e2.employeeid
;

In [ ]:
-- Puzzle 10 DDL
CREATE OR REPLACE TEMPORARY TABLE SampleData
    (
    IntegerValue  INTEGER NOT NULL
    )
;

INSERT INTO SampleData (IntegerValue)
VALUES
    (5),(6),(10),(10),(13),(14),(17),(20),(81),(90),(76)
;

In [ ]:
-- Puzzle 10
SELECT
    AVG(sd.integervalue)
    ,MEDIAN(sd.integervalue)
    ,MODE(sd.integervalue)
    ,MAX(sd.integervalue)-MIN(sd.integervalue)
FROM
    sampledata AS sd
;

In [ ]:
-- Puzzle 11 DDL
CREATE OR REPLACE TEMPORARY TABLE TestCases
    (
    TestCase  VARCHAR(1)
    )
;

INSERT INTO TestCases (TestCase)
VALUES
    ('A')
    ,('B')
    ,('C')
;

In [ ]:
-- Puzzle 11
;WITH base AS
    (
    SELECT
        tc.testcase
        ,ARRAY_CONSTRUCT(tc.testcase) AS testcase_array
    FROM
        testcases AS tc

    UNION ALL

    SELECT
        tc.testcase
        ,ARRAY_APPEND(b.testcase_array, tc.testcase) AS testcase_array
    FROM
        base AS b
        JOIN testcases AS tc ON tc.testcase<>b.testcase AND NOT ARRAY_CONTAINS(tc.testcase::VARIANT, b.testcase_array)
    )

SELECT
    ARRAY_TO_STRING(b.testcase_array, ',') AS testcase
FROM
    base AS b
WHERE 1=1
    AND ARRAY_SIZE(b.testcase_array)=(SELECT COUNT(*) FROM testcases AS tc)
;

In [ ]:
-- Puzzle 12 DDL
CREATE OR REPLACE TEMPORARY TABLE ProcessLog
    (
    WorkFlow       VARCHAR(100),
    ExecutionDate  DATE
    )
;

INSERT INTO ProcessLog (WorkFlow, ExecutionDate)
VALUES
    ('Alpha','6/01/2018'),('Alpha','6/14/2018'),('Alpha','6/15/2018'),
    ('Bravo','6/1/2018'),('Bravo','6/2/2018'),('Bravo','6/19/2018'),
    ('Charlie','6/1/2018'),('Charlie','6/15/2018'),('Charlie','6/30/2018')
;

In [ ]:
-- Puzzle 12
SELECT
    f.workflow
    ,FLOOR(AVG(f.dd)) AS average_days
FROM
    (
    SELECT
        *
        ,LEAD(pl.executiondate) OVER(PARTITION BY pl.workflow ORDER BY pl.executiondate) AS lead_executiondate
        ,DATEDIFF(DAY, pl.executiondate, lead_executiondate) AS dd
    FROM
        processlog AS pl
    ) AS f
GROUP BY ALL
;

In [ ]:
-- Puzzle 13 DDL
CREATE OR REPLACE TEMPORARY TABLE Inventory
    (
    InventoryDate       DATE,
    QuantityAdjustment  INTEGER NOT NULL
    )
;

INSERT INTO Inventory (InventoryDate, QuantityAdjustment)
VALUES
    ('7/1/2018',100)
    ,('7/2/2018',75)
    ,('7/3/2018',-150)
    ,('7/4/2018',50)
    ,('7/5/2018',-100)
;

In [ ]:
-- Puzzle 13
SELECT
    i.inventorydate
    ,i.quantityadjustment
    ,SUM(i.quantityadjustment) OVER(ORDER BY i.inventorydate) AS inventory
FROM
    inventory AS i
ORDER BY
    i.inventorydate
;

In [ ]:
-- Puzzle 14 DDL
CREATE OR REPLACE TEMPORARY TABLE ProcessLog
    (
    Workflow    VARCHAR(100),
    StepNumber  INTEGER,
    RunStatus   VARCHAR(100) NOT NULL
    )
;

INSERT INTO ProcessLog (Workflow, StepNumber, RunStatus)
VALUES
    ('Alpha',1,'Error'),('Alpha',2,'Complete'),('Alpha',3,'Running'),
    ('Bravo',1,'Complete'),('Bravo',2,'Complete'),
    ('Charlie',1,'Running'),('Charlie',2,'Running'),
    ('Delta',1,'Error'),('Delta',2,'Error'),
    ('Echo',1,'Running'),('Echo',2,'Complete')
;

In [ ]:
-- Puzzle 14
SELECT DISTINCT
    f.workflow
    ,CASE
        WHEN f.ct_distinct_status_workflow=1 THEN f.runstatus
        WHEN f.ct_error_workflow>0 THEN 'Indeterminate'
        ELSE 'Running'
    END AS status
FROM
    (
    SELECT
        *
        ,COUNT(*) OVER (PARTITION BY pl.workflow) AS ct_row_workflow
        ,COUNT(DISTINCT pl.runstatus) OVER (PARTITION BY pl.workflow) AS ct_distinct_status_workflow
        ,COUNT_IF(pl.runstatus='Error') OVER (PARTITION BY pl.workflow) AS ct_error_workflow
    FROM
        processlog AS pl
    ) AS f
;

In [ ]:
-- Puzzle 15 DDL
CREATE OR REPLACE TEMPORARY TABLE DMLTable
    (
    SequenceNumber  INTEGER PRIMARY KEY,
    String          VARCHAR(100) NOT NULL
    )
;

INSERT INTO DMLTable (SequenceNumber, String)
VALUES
    (1,'SELECT'),
    (2,'Product,'),
    (3,'UnitPrice,'),
    (4,'EffectiveDate'),
    (5,'FROM'),
    (6,'Products'),
    (7,'WHERE'),
    (8,'UnitPrice'),
    (9,'> 100')
;

In [ ]:
-- Puzzle 15
SELECT
    LISTAGG(d.string, ' ') WITHIN GROUP (ORDER BY d.sequencenumber) AS syntax
FROM
    dmltable AS d
;

In [ ]:
-- Puzzle 16 DDL
CREATE OR REPLACE TEMPORARY TABLE PlayerScores
    (
    PlayerA  INTEGER,
    PlayerB  INTEGER,
    Score    INTEGER NOT NULL
    )
;

INSERT INTO PlayerScores (PlayerA, PlayerB, Score)
VALUES
    (1001,2002,150)
    ,(3003,4004,15)
    ,(4004,3003,125)
;

In [ ]:
-- Puzzle 16
SELECT
    p1.playera
    ,p1.playerb
    ,(p1.score + ZEROIFNULL(p2.score)) AS score
FROM
    playerscores AS p1
    LEFT JOIN playerscores AS p2 ON p2.playerb=p1.playera AND p2.playera=p1.playerb
WHERE 1=1
    AND p1.playera < p1.playerb
;

In [ ]:
-- Puzzle 17 DDL
CREATE OR REPLACE TEMPORARY TABLE Ungroup
    (
    ProductDescription  VARCHAR(100),
    Quantity            INTEGER NOT NULL
    )
;

INSERT INTO Ungroup (ProductDescription, Quantity)
VALUES
    ('Pencil',3)
    ,('Eraser',4)
    ,('Notebook',2)
;

In [ ]:
-- Puzzle 17
;WITH numbers AS
    (
    SELECT
        seq4()+1 AS num
    FROM
        TABLE(GENERATOR(ROWCOUNT => 10)) AS t
    )

SELECT
    u.productdescription
    ,1 AS quantity
FROM
    ungroup AS u
    JOIN numbers AS n ON n.num<=u.quantity
ORDER BY
    u.productdescription
;

In [ ]:
-- Puzzle 18 DDL
CREATE OR REPLACE TEMPORARY TABLE SeatingChart
    (
    SeatNumber  INTEGER
    )
;

INSERT INTO SeatingChart (SeatNumber)
VALUES
    (7),(13),(14),(15),(27),(28),(29),(30),(31),(32),(33),(34),(35),(52),(53),(54)
;

In [ ]:
-- Puzzle 18
;WITH numbers AS
    (
    SELECT
        seq4()+1 AS num
    FROM
        TABLE(GENERATOR(ROWCOUNT => 100)) AS t
    )
,base AS
    (
    SELECT
        *
        ,LEAD(s.seatnumber) OVER(ORDER BY s.seatnumber) AS lead_seatnumber
        ,(lead_seatnumber-s.seatnumber) AS diff
        ,MIN(s.seatnumber) OVER() AS min_seatnumber
    FROM
        seatingchart AS s
    QUALIFY
        diff > 1
    )
,final AS
    (
    SELECT
        b.seatnumber+1 AS gapstart
        ,b.lead_seatnumber-1 AS gapend
    FROM
        base AS b
    
    UNION ALL
    
    SELECT
        1 AS gapstart
        ,b.seatnumber-1 AS gapend
    FROM
        base AS b
    WHERE 1=1
        AND b.min_seatnumber<>1
        AND b.min_seatnumber=b.seatnumber
    
    ORDER BY
        gapstart
    )

-- -- part 1
-- SELECT
--     *
-- FROM
--     final AS f
-- ;

-- -- part 2
-- SELECT
--     COUNT(*) AS totalmissingnumbers
-- FROM
--     numbers AS n
--     JOIN final AS f ON n.num BETWEEN f.gapstart AND f.gapend
-- ;

-- part 3
SELECT
    IFF(MOD(s.seatnumber, 2)=0,'Even Numbers','Odd Numbers') AS type
    ,COUNT(*) AS ct
FROM
    seatingchart AS s
GROUP BY ALL
ORDER BY
    type
;

In [ ]:
-- Puzzle 19 DDL
CREATE OR REPLACE TEMPORARY TABLE TimePeriods
    (
    StartDate  DATE,
    EndDate    DATE
    )
;

INSERT INTO TimePeriods (StartDate, EndDate)
VALUES
    ('1/1/2018','1/5/2018'),
    ('1/3/2018','1/9/2018'),
    ('1/10/2018','1/11/2018'),
    ('1/12/2018','1/16/2018'),
    ('1/15/2018','1/19/2018')
;

In [ ]:
-- Puzzle 19
;WITH base AS
    (
    SELECT
        *
        ,LEAD(t.startdate) OVER(ORDER BY t.startdate) AS lead_startdate
        ,LAG(t.enddate) OVER(ORDER BY t.enddate) AS lag_enddate
        ,lead_startdate<t.enddate AS has_overlap
        ,(lag_enddate<t.startdate) AND (lead_startdate>t.enddate) AS no_overlap
    FROM
        timeperiods AS t
    )

SELECT
    b1.startdate
    ,IFF(b1.has_overlap,b2.enddate,b1.enddate) AS enddate
FROM
    base AS b1
    LEFT JOIN base AS b2 ON b2.startdate=b1.lead_startdate AND b1.has_overlap
WHERE 1=1
    AND (b1.has_overlap OR b1.no_overlap)
ORDER BY
    b1.startdate
;

In [ ]:
-- Puzzle 20 DDL
CREATE OR REPLACE TEMPORARY TABLE ValidPrices
    (
    ProductID      INTEGER,
    UnitPrice      DECIMAL(38,2),
    EffectiveDate  DATE
    )
;

INSERT INTO ValidPrices (ProductID, UnitPrice, EffectiveDate)
VALUES
    (1001,1.99,'1/01/2018'),
    (1001,2.99,'4/15/2018'),
    (1001,3.99,'6/8/2018'),
    (2002,1.99,'4/17/2018'),
    (2002,2.99,'5/19/2018')
;

In [ ]:
-- Puzzle 20 (Solution 1)
SELECT
    *
FROM
    validprices AS v
QUALIFY
    ROW_NUMBER() OVER(PARTITION BY v.productid ORDER BY v.effectivedate DESC)=1
;

In [ ]:
-- Puzzle 20 (Solution 2)
SELECT
    v.productid
    ,MAX_BY(v.effectivedate, v.effectivedate) AS effectivedate
    ,MAX_BY(v.unitprice, v.effectivedate) AS unitprice
FROM
    validprices AS v
GROUP BY ALL
ORDER BY
    v.productid
;